In [18]:
"""cima_client.py
====================
Cliente asíncrono minimalista para invocar **todas** las habilidades/endpoint
oficiales de la API REST de CIMA (AEMPS) y devolver la respuesta *raw* (dict,
list, str o None).

Cada endpoint está expuesto como función independiente para poder reutilizarla
fácilmente desde otros scripts o cuadernos Jupyter.

• Probado con Python 3.12, httpx 0.27   —   2025‑05‑04.

Ejemplo rápido (CLI):
    python -m asyncio -c "import asyncio, cima_raw_client as c; print(asyncio.run(c.medicamento(cn='608679')))"
"""
from __future__ import annotations

import asyncio
import json
from datetime import date
from pathlib import Path
from typing import Any, Dict, Optional

import httpx

BASE_URL = "https://cima.aemps.es/cima/rest"
TIMEOUT = httpx.Timeout(15)

# ---------------------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------------------

def _clean(params: Dict[str, Any] | None) -> Dict[str, Any] | None:
    """Elimina claves con valor `None` para formar el querystring."""
    if not params:
        return None
    return {k: v for k, v in params.items() if v is not None}


async def _request(
    method: str,
    path: str,
    *,
    params: Dict[str, Any] | None = None,
    json_body: Any | None = None,
    client: httpx.AsyncClient | None = None,
) -> Any | None:
    """Lanza la petición y devuelve datos parseados o str si no es JSON."""
    owns_client = client is None
    if owns_client:
        client = httpx.AsyncClient(timeout=TIMEOUT)

    try:
        resp = await client.request(method, f"{BASE_URL}/{path}", params=_clean(params), json=json_body)
        resp.raise_for_status()

        # Cuerpo vacío
        if not resp.content:
            return None

        # Intentamos JSON; si falla devolvemos text
        try:
            return resp.json()
        except (json.JSONDecodeError, ValueError):
            return resp.text
    finally:
        if owns_client:
            await client.aclose()


# ---------------------------------------------------------------------------
# 1. Medicamentos
# ---------------------------------------------------------------------------
async def medicamentos(
    *,
    nombre: str | None = None,
    laboratorio: str | None = None,
    practiv1: str | None = None,
    practiv2: str | None = None,
    idpractiv1: str | None = None,
    idpractiv2: str | None = None,
    cn: str | None = None,
    atc: str | None = None,
    nregistro: str | None = None,
    npactiv: int | None = None,
    triangulo: int | None = None,
    huerfano: int | None = None,
    biosimilar: int | None = None,
    sust: int | None = None,
    vmp: str | None = None,
    comerc: int | None = None,
    autorizados: int | None = None,
    receta: int | None = None,
    estupefaciente: int | None = None,
    psicotropo: int | None = None,
    estuopsico: int | None = None,
    pagina: int | None = None,
) -> Any | None:
    """GET /medicamentos – Lista paginada con múltiples filtros."""
    return await _request(
        "GET",
        "medicamentos",
        params=locals(),
    )


async def medicamento(*, cn: str | None = None, nregistro: str | None = None) -> Any | None:
    """GET /medicamento – Ficha completa del medicamento (cn o nregistro)."""
    if not (cn or nregistro):
        raise ValueError("Se requiere 'cn' o 'nregistro'.")
    return await _request("GET", "medicamento", params=locals())


# ---------------------------------------------------------------------------
# 2. Búsqueda en ficha técnica
# ---------------------------------------------------------------------------
async def buscar_en_ficha_tecnica(reglas: list[dict[str, Any]]) -> Any | None:
    """POST /buscarEnFichaTecnica – Array de reglas: seccion, texto, contiene(0|1)."""
    if not reglas:
        raise ValueError("Debe proporcionar al menos una regla de búsqueda.")
    return await _request("POST", "buscarEnFichaTecnica", json_body=reglas)


# ---------------------------------------------------------------------------
# 3. Presentaciones
# ---------------------------------------------------------------------------
async def presentaciones(
    *,
    cn: str | None = None,
    nregistro: str | None = None,
    vmp: str | None = None,
    vmpp: str | None = None,
    idpractiv1: str | None = None,
    comerc: int | None = None,
    estupefaciente: int | None = None,
    psicotropo: int | None = None,
    estuopsico: int | None = None,
    pagina: int | None = None,
) -> Any | None:
    """GET /presentaciones – Listado de presentaciones."""
    return await _request("GET", "presentaciones", params=locals())


async def presentacion(cn: str) -> Any | None:
    """GET /presentacion/{cn} – Detalle de una presentación concreta."""
    return await _request("GET", f"presentacion/{cn}")


# ---------------------------------------------------------------------------
# 4. Descripción clínica (VMP/VMPP)
# ---------------------------------------------------------------------------
async def vmpp(
    *,
    practiv1: str | None = None,
    idpractiv1: str | None = None,
    dosis: str | None = None,
    forma: str | None = None,
    atc: str | None = None,
    nombre: str | None = None,
    modoArbol: int | None = None,
    pagina: int | None = None,
) -> Any | None:
    """GET /vmpp – Devuelve VMP/VMPP filtrados."""
    return await _request("GET", "vmpp", params=locals())


# ---------------------------------------------------------------------------
# 5. Maestras
# ---------------------------------------------------------------------------
async def maestras(
    *,
    maestra: int | None = None,
    nombre: str | None = None,
    id: str | None = None,
    codigo: str | None = None,
    estupefaciente: int | None = None,
    psicotropo: int | None = None,
    estuopsico: int | None = None,
    enuso: int | None = None,
    pagina: int | None = None,
) -> Any | None:
    """GET /maestras – Catálogos de laboratorios, ATC, formas, etc."""
    return await _request("GET", "maestras", params=locals())


# ---------------------------------------------------------------------------
# 6. Registro de cambios
# ---------------------------------------------------------------------------
async def registro_cambios(
    *,
    fecha: str | None = None,
    nregistro: str | None = None,
    metodo: str = "GET",  # Puede ser "GET" o "POST" según se prefiera
) -> Any | None:
    """GET/POST /registroCambios – Seguimiento de altas/bajas/modificaciones."""
    if fecha is None:
        fecha = date.today().strftime("%d/%m/%Y")
    if metodo.upper() == "POST":
        return await _request("POST", "registroCambios", json_body={"fecha": fecha, "nregistro": nregistro})
    return await _request("GET", "registroCambios", params={"fecha": fecha, "nregistro": nregistro})


# ---------------------------------------------------------------------------
# 7. Problemas de suministro
# ---------------------------------------------------------------------------
async def psuministro(cn: str | None = None) -> Any | None:
    """GET /psuministro o /psuministro/{cn}."""
    path = "psuministro" if cn is None else f"psuministro/{cn}"
    return await _request("GET", path)


# ---------------------------------------------------------------------------
# 8. Documentos segmentados
# ---------------------------------------------------------------------------
async def doc_secciones(tipo_doc: int, *, nregistro: str | None = None, cn: str | None = None) -> Any | None:
    """GET /docSegmentado/secciones/{tipo_doc} – Lista metadatos secciones."""
    if not (nregistro or cn):
        raise ValueError("Se requiere 'nregistro' o 'cn'.")
    return await _request("GET", f"docSegmentado/secciones/{tipo_doc}", params=_clean({"nregistro": nregistro, "cn": cn}))


async def doc_contenido(
    tipo_doc: int,
    *,
    nregistro: str | None = None,
    cn: str | None = None,
    seccion: str | None = None,
) -> Any | None:
    """GET /docSegmentado/contenido/{tipo_doc} – Contenido HTML o JSON de secciones."""
    if not (nregistro or cn):
        raise ValueError("Se requiere 'nregistro' o 'cn'.")
    return await _request(
        "GET",
        f"docSegmentado/contenido/{tipo_doc}",
        params=_clean({"nregistro": nregistro, "cn": cn, "seccion": seccion}),
    )


# ---------------------------------------------------------------------------
# __main__ – demostración rápida (CLI)
# ---------------------------------------------------------------------------
if __name__ == "__main__":
    async def demo():
        print(json.dumps(await medicamento(cn="679688"), indent=2, ensure_ascii=False))

    # Maneja ejecución en entornos con loop activo (Jupyter) de forma segura
    try:
        asyncio.run(demo())
    except RuntimeError as exc:
        if "asyncio.run()" in str(exc):
            import nest_asyncio

            nest_asyncio.apply()
            asyncio.get_event_loop().run_until_complete(demo())
        else:
            raise


{
  "nregistro": "61068",
  "nombre": "CINFATOS COMPLEX suspensión oral",
  "pactivos": "PARACETAMOL, DEXTROMETORFANO HIDROBROMURO, PSEUDOEFEDRINA HIDROCLORURO",
  "labtitular": "Laboratorios Cinfa S.A.",
  "labcomercializador": "Laboratorios Cinfa S.A.",
  "cpresc": "Sin Receta",
  "estado": {
    "aut": 825634800000
  },
  "comerc": true,
  "receta": false,
  "generico": false,
  "conduc": false,
  "triangulo": false,
  "huerfano": false,
  "biosimilar": false,
  "nosustituible": {
    "id": 0,
    "nombre": "N/A"
  },
  "psum": false,
  "notas": true,
  "materialesInf": false,
  "ema": false,
  "docs": [
    {
      "tipo": 1,
      "url": "https://cima.aemps.es/cima/pdfs/ft/61068/FT_61068.pdf",
      "urlHtml": "https://cima.aemps.es/cima/dochtml/ft/61068/FT_61068.html",
      "secc": true,
      "fecha": 1739839570000
    },
    {
      "tipo": 2,
      "url": "https://cima.aemps.es/cima/pdfs/p/61068/P_61068.pdf",
      "urlHtml": "https://cima.aemps.es/cima/dochtml/p/61068/P_6106

In [23]:
import asyncio
from pathlib import Path
import httpx

# Asume que la función `medicamento` ya está definida en el entorno (Jupyter Notebook).

# Mapeo de tipos de documento a sus códigos en la API
_DOC_TYPE_MAP = {
    'ft': 1,   # Ficha Técnica
    'p': 2,    # Prospecto/Prescripción
    'ipt': 3,  # Informe de Posicionamiento Terapéutico
}
# Tipos de imágenes en alta resolución disponibles en CIMA
_IMG_FULL_TYPES = ['formafarmac', 'materialas']
# Encabezados simulando navegador para esquivar bloqueos
_DEFAULT_HEADERS = {'User-Agent': 'Mozilla/5.0'}

async def download_docs(
    *,
    cn: str | None = None,
    nregistro: str | None = None,
    tipos: list[str] = ['ft', 'p', 'ipt'],
    client: httpx.AsyncClient | None = None,
) -> None:
    """
    Descarga los documentos PDF solicitados para un medicamento.

    Ejemplo en Jupyter:
        await download_docs(cn='608679', tipos=['ft', 'p'])
    """
    if not (cn or nregistro):
        raise ValueError("Se requiere 'cn' o 'nregistro'.")

    med = await medicamento(cn=cn, nregistro=nregistro)
    if not med or not isinstance(med, dict):
        print(f"No se obtuvo información del medicamento (cn={cn}, nregistro={nregistro}).")
        return

    docs = med.get('docs') or []
    if not docs:
        print("No hay documentos disponibles para descarga.")
        return

    owns_client = client is None
    if owns_client:
        client = httpx.AsyncClient(timeout=httpx.Timeout(15), headers=_DEFAULT_HEADERS)

    try:
        for tipo in tipos:
            code = _DOC_TYPE_MAP.get(tipo.lower())
            if not code:
                print(f"Tipo de documento desconocido: {tipo}")
                continue

            dest_dir = Path('data/pdf') / tipo.lower()
            dest_dir.mkdir(parents=True, exist_ok=True)

            for doc in docs:
                if doc.get('tipo') == code:
                    url = doc.get('url')
                    filename = Path(url or '').name
                    filepath = dest_dir / filename
                    try:
                        resp = await client.get(url, follow_redirects=True)
                        resp.raise_for_status()
                        filepath.write_bytes(resp.content)
                        print(f"Descargado {tipo.upper()} -> {filepath}")
                    except httpx.HTTPStatusError as e:
                        print(f"Error HTTP {e.response.status_code} descargando {url}")
                    except Exception as e:
                        print(f"Error descargando {url}: {e}")
    finally:
        if owns_client:
            await client.aclose()

async def download_images(
    *,
    cn: str | None = None,
    nregistro: str | None = None,
    client: httpx.AsyncClient | None = None,
) -> None:
    """
    Descarga las imágenes en alta resolución (formafarmac y materialas) para un medicamento.

    Intenta primero con el nregistro obtenido de la llamada `medicamento`, y si no está disponible o falla, prueba con el CN.

    Ejemplo en Jupyter:
        await download_images(cn='608679')
        await download_images(nregistro='65679')
    """
    if not (cn or nregistro):
        raise ValueError("Se requiere 'cn' o 'nregistro'.")

    # Obtener nregistro desde API si no fue proporcionado
    med = await medicamento(cn=cn, nregistro=nregistro)
    if not med or not isinstance(med, dict):
        print(f"No se obtuvo información del medicamento (cn={cn}, nregistro={nregistro}).")
        return
    reg_api = med.get('nregistro')

    # Prioridad de identificación: parámetro nregistro, luego valor de API, luego cn
    ids_to_try = []
    if nregistro:
        ids_to_try.append(str(nregistro))
    elif reg_api:
        ids_to_try.append(str(reg_api))
    if cn and str(cn) not in ids_to_try:
        ids_to_try.append(str(cn))

    base = "https://cima.aemps.es/cima/fotos/full"
    owns_client = client is None
    if owns_client:
        client = httpx.AsyncClient(timeout=httpx.Timeout(15), headers=_DEFAULT_HEADERS)

    try:
        for img_type in _IMG_FULL_TYPES:
            success = False
            for identifier in ids_to_try:
                url = f"{base}/{img_type}/{identifier}/{identifier}_{img_type}.jpg"
                dest_dir = Path('data/img') / img_type
                dest_dir.mkdir(parents=True, exist_ok=True)
                filepath = dest_dir / f"{identifier}_{img_type}.jpg"
                try:
                    resp = await client.get(url, follow_redirects=True)
                    resp.raise_for_status()
                    filepath.write_bytes(resp.content)
                    print(f"Descargada imagen {img_type} para ID={identifier} -> {filepath}")
                    success = True
                    break
                except httpx.HTTPStatusError as e:
                    if e.response.status_code != 404:
                        print(f"Error HTTP {e.response.status_code} al descargar imagen {url}")
                except Exception as e:
                    print(f"Error descargando imagen {url}: {e}")
            if not success:
                print(f"No se encontró ninguna imagen {img_type} para IDs: {ids_to_try}")
    finally:
        if owns_client:
            await client.aclose()


await download_docs(cn='679688', tipos=['ft', 'p', 'ipt'])
await download_images(cn='679688')


Descargado FT -> data\pdf\ft\FT_61068.pdf
Descargado P -> data\pdf\p\P_61068.pdf
Descargada imagen formafarmac para ID=61068 -> data\img\formafarmac\61068_formafarmac.jpg
Descargada imagen materialas para ID=61068 -> data\img\materialas\61068_materialas.jpg


In [ ]:
# En cima_client.py (módulo de herramientas para mcp_aemps_server)
import asyncio
from pathlib import Path
import httpx

def _ensure_dir(path: Path) -> None:
    """Crea el directorio si no existe."""
    path.mkdir(parents=True, exist_ok=True)

# Mapas de tipos
_DOC_TYPE_MAP = {
    'ft': 1,
    'p': 2,
    'ipt': 3,
}
_IMG_FULL_TYPES = ['formafarmac', 'materialas']
_DEFAULT_HEADERS = {'User-Agent': 'Mozilla/5.0'}


async def download_docs_tool(
    cn: str | None = None,
    nregistro: str | None = None,
    tipos: list[str] = ['ft', 'p', 'ipt'],
    base_dir: str = 'data/pdf',
    timeout: int = 15,
) -> list[str]:
    """
    Tool: Descarga PDFs de CIMA.
    Args:
      cn: Código Nacional.
      nregistro: Número de registro.
      tipos: ['ft','p','ipt'].
      base_dir: Carpeta raíz para guardar.
      timeout: segundos de timeout HTTP.
    Returns:
      Lista de rutas de archivos descargados.
    """
    med = await medicamento(cn=cn, nregistro=nregistro)
    if not med or not isinstance(med, dict):
        return []
    docs = med.get('docs') or []
    if not docs:
        return []

    client = httpx.AsyncClient(timeout=httpx.Timeout(timeout), headers=_DEFAULT_HEADERS)
    downloaded = []
    try:
        for tipo in tipos:
            code = _DOC_TYPE_MAP.get(tipo.lower())
            if not code:
                continue
            dest_dir = Path(base_dir) / tipo.lower()
            _ensure_dir(dest_dir)
            for doc in docs:
                if doc.get('tipo') == code and doc.get('url'):
                    url = doc['url']
                    resp = await client.get(url, follow_redirects=True)
                    resp.raise_for_status()
                    filepath = dest_dir / Path(url).name
                    filepath.write_bytes(resp.content)
                    downloaded.append(str(filepath))
    finally:
        await client.aclose()
    return downloaded


async def download_images_tool(
    cn: str | None = None,
    nregistro: str | None = None,
    base_dir: str = 'data/img',
    timeout: int = 15,
) -> list[str]:
    """
    Tool: Descarga imágenes full-resolution.
    Args:
      cn: Código Nacional.
      nregistro: Número de registro.
      base_dir: Carpeta para imágenes.
      timeout: segundos de timeout HTTP.
    Returns:
      Lista de rutas de imágenes descargadas.
    """
    med = await medicamento(cn=cn, nregistro=nregistro)
    if not med or not isinstance(med, dict):
        return []
    reg_api = med.get('nregistro')
    # Build identifiers sequence
    ids_to_try = []
    if nregistro:
        ids_to_try.append(str(nregistro))
    elif reg_api:
        ids_to_try.append(str(reg_api))
    if cn and str(cn) not in ids_to_try:
        ids_to_try.append(str(cn))

    client = httpx.AsyncClient(timeout=httpx.Timeout(timeout), headers=_DEFAULT_HEADERS)
    downloaded = []
    base = "https://cima.aemps.es/cima/fotos/full"
    try:
        for img_type in _IMG_FULL_TYPES:
            for identifier in ids_to_try:
                url = f"{base}/{img_type}/{identifier}/{identifier}_{img_type}.jpg"
                resp = await client.get(url, follow_redirects=True)
                if resp.status_code == 404:
                    continue
                resp.raise_for_status()
                dest_dir = Path(base_dir) / img_type
                _ensure_dir(dest_dir)
                filepath = dest_dir / f"{identifier}_{img_type}.jpg"
                filepath.write_bytes(resp.content)
                downloaded.append(str(filepath))
                break
    finally:
        await client.aclose()
    return downloaded
